In [2]:
import pickle
import sys
from collections import Counter
import numpy as np
from scipy import spatial
from pandas import DataFrame

sys.path.append("/home/jichao/Desktop/reddit/language_model")
from language_model import *

tokens_fem2men_pre_cp = pickle.load(open("tokens_fem2men_pre_cp.pickle"))
tokens_fem2men_post_cp_fem = pickle.load(open("tokens_fem2men_post_cp_fem.pickle"))
tokens_fem2men_post_cp_men = pickle.load(open("tokens_fem2men_post_cp_men.pickle"))
tokens_fem_only = pickle.load(open("tokens_fem_only.pickle"))

d = [tokens_fem2men_pre_cp,
    tokens_fem2men_post_cp_fem,
    tokens_fem2men_post_cp_men,
    tokens_fem_only]

logprob = np.empty((4, 4))

for i in range(len(d)):
    train = d[i]
    train_tokens = set(list(itertools.chain(*train)))

    for j in range(len(d)):
        if j != i:
            test = d[j]
            test_tokens = list(itertools.chain(*test))

            bigram = Bigram(train, special_token=True)
            oov = filter(lambda token: token not in train_tokens, test_tokens)
            oov = len(set(oov))

            bigram.fit(oov)
            data = map(lambda tokens: bigram.predict(tokens), test)
            logprob[i][j] = np.mean(data)
        else:
            logprob[i][j] = np.nan

logprob = DataFrame(logprob,
                    index=["F2M_PreCP_F", "F2M_PostCP_F", "F2M_PostCP_M", "F_Only"],
                    columns=["F2M_PreCP_F", "F2M_PostCP_F", "F2M_PostCP_M", "F_Only"])
logprob.index.name = "Train"
logprob.columns.name = "Test"
logprob


Test,F2M_PreCP_F,F2M_PostCP_F,F2M_PostCP_M,F_Only
Train,,,,
F2M_PreCP_F,NaN,-7.187181,-7.214050,-7.126711
F2M_PostCP_F,-6.920193,NaN,-7.109825,-7.049650
F2M_PostCP_M,-6.879710,-7.028195,NaN,-6.992669
F_Only,-6.786466,-6.947195,-6.993046,NaN
